# Inspect GLASS models and generate synthetic images

## Imports

In [1]:
import sys
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
root = "/Users/phdenzel/gleam"
sys.path.append(root)
import gleam
from gleam.lensobject import LensObject
from gleam.utils.lensing import LensModel
from gleam.reconsrc import ReconSrc, run_model
from gleam.utils.plotting import kappa_map_plot, kappa_profiles_plot
from gleam.utils.plotting import arrival_time_surface_plot
from gleam.utils.plotting import plot_scalebar, plot_labelbox
from gleam.utils.rgb_map import radial_mask
import gleam.utils.colors as gcl
gcl.GLEAMcmaps.register_all()


## Load data files

In [2]:
#        ACS          ACS(WFC3)        ACS(WFC3)      WFC3(UNK)
#        ACS(WFC3)    SBC              WFC3           ACS(GAIA)
objs = ['B1608+656', 'DESJ0408-5354', 'HE0435-1223', 'PG1115+080',
        'RXJ0911+0551', 'RXJ1131-1231', 'SDSSJ1004+4112', 'WFIJ2033-4723']
objidx = 3
sigf = 600     #   80,  100,   60,  600,  140, 4000,   80,   80
lens = objs[objidx]
print(lens)


PG1115+080


In [3]:
fitsdir = 'data/delay_qsos/'
fitsfile = fitsdir + '{}.fits'.format(lens)
print(fitsfile)

jsondir = 'jsons/'
jsonfile = jsondir + '{}.json'.format(lens)
print(jsonfile)


data/delay_qsos/PG1115+080.fits
jsons/PG1115+080.json


In [4]:
with open(jsonfile) as f:
    lo = LensObject.from_json(f)
lo.squarify(0)
print(lo.__v__)


Properties of that field are unknown [None]
filename            	PG1115+080.json
filepath            	/Users/phdenzel/delays/jsons/PG1115+080.json
band                	
naxis1              	99
naxis2              	99
naxis_plus          	None
refval              	[169.57094740092, 7.766067434866]
refpx               	[36.0, 41.0]
center              	<169.5705, 7.7664>
px2deg              	[3.562500079472802e-05, 3.5625000794728505e-05]
px2arcsec           	[0.12825000286102087, 0.12825000286102262]
megacam_range       	None
field               	None
photzp              	None
mag_formula         	<function mag_formula at 0x7f8b5acce848>
roi                 	ROISelector(99, 99)
lens                	None
srcimgs             	[]
zl                  	0.5
zs                  	3.0
mapr                	6.3484
tdelay              	None
tderr               	None
light_model         	{}
stel_mass           	None
glscfactory         	GLSCFactory(zl, srcimgs, parity, tderr...)
finder              

In [5]:
statedir = 'states/'
statefiles = ['11doubles_dg45.state',
              '11doubles_dg60.state', '11doubles_CMB_dg60.state', '11doubles_SNeIa_dg60.state',
              '7quads_dg45.state', '7quads_CMB_dg45.state', '7quads_SNeIa_dg45.state',
              '7quads_dg60.state', '7quads_CMB_dg60.state', '7quads_SNeIa_dg60.state', 
              'all_dg60.state', 'all_SNeIa_dg60.state']
statefile = statefiles[7]  # 7  8/9 5/6
print(statefile)


7quads_dg60.state


In [6]:
lm = LensModel(statedir+statefile)
lm.obj_idx = objidx
print(lm.__v__)


filename            	7quads_dg60.state
filepath            	states/7quads_dg60.state
N                   	1000
pixrad              	8
maprad              	1.67273244599
pixel_size          	0.0393584104938
kappa               	None
minima              	[[ 0.11439883300091856  1.3727859960110342 ]
 [-0.7864919768813123  -0.872291101632015  ]]
saddle_points       	[[-1.058189205258495   -0.4289956237534523 ]
 [ 0.8579912475068856  -0.45759533200368047]]
maxima              	[]
zl                  	0.311
zs                  	1.735
N_obj               	8
N_src               	1
obj_idx             	3
src_idx             	0


## Model plots

### Kappa maps

In [ ]:
#%%script false
# for i in range(lm.N_obj):
#     lm.obj_idx = i
i = objidx
kappa_map_plot(lm, obj_index=i, extent=lm.extent, contours=True, levels=7, delta=0.1, label=lm.obj_name)
plot_scalebar(R=lm.maprad, length=max(int(lm.maprad/2), 1))
plt.axis('off')
plt.gcf().axes[0].get_xaxis().set_visible(False)
plt.gcf().axes[0].get_yaxis().set_visible(False)
plt.savefig('results/{}_{}_kappa.pdf'.format(lm.obj_name, lm.filename.replace('.state', '')), transparent=True, bbox_inches='tight', pad_inches=0)
plt.close()
plt.show()


### Kappa profiles

In [ ]:
#%%script false
# for i in range(lm.N_obj):
#     lm.obj_idx = i
i = objidx
kappa_profiles_plot(lm, obj_index=i, ensemble_average=True, refined=True, interpolate=150, levels=10,
                    as_range=True, maprad=lm.maprad, pixrad=lm.pixrad,
                    adjust_limits=True, annotation_color='white', label_axes=True, fontsize=22)
plt.tight_layout()
plt.savefig('results/{}_{}_profiles.pdf'.format(lm.obj_name, lm.filename.replace('.state', '')), transparent=True, bbox_inches='tight', pad_inches=0)
# plt.close()
plt.show()


In [ ]:
%%script false
fig, axes = plt.subplots(4, 2, sharex=False, sharey=True, figsize=(7, 11))
for i in range(len(objs)):
    lm.obj_idx = i
    print(i)
    yextent = [0.65, 3.00]
    plt.sca(axes[i // 2][i % 2])
    plots, _, _ = kappa_profiles_plot(lm, obj_index=i, ensemble_average=True, refined=True,
                                      interpolate=150, levels=10, as_range=True, maprad=lm.maprad,
                                      pixrad=lm.pixrad, adjust_limits=True, annotation_color='white',
                                      einstein_radius_indicator=True, kappa1_line=True,
                                      label_axes=False, fontsize=22)
    xlim = list(plt.gca().axes.get_xlim())
    plt.contourf(np.ones((4, 4))*plots[0].levels[0], extent=xlim+yextent,
                 cmap='agaveglitch', levels=plots[0].levels, zorder=-99)
    plot_labelbox(objs[i], position='top right', padding=(0.03, 0.04), color='white', fontsize=8)
    plt.ylim(*yextent)
    # plt.rcParams['mathtext.fontset'] = 'stixsans'
    if (i % 2) == 0:
        if (i // 2) == 2:
            axes[i // 2][i % 2].set_ylabel(r'$\mathsf{\kappa}_{<\mathsf{R}}$', fontsize=22)
    if (i // 2) > 2:
        if (i % 2) == 0:
            axes[i // 2][i % 2].set_xlabel(r'R [arcsec]', fontsize=16)
#     else:
#         axes[i // 2][i % 2].set_xticklabels([""]*len(axes[i // 2][i % 2].axes.get_xticklabels()))
    axes[i // 2][i % 2].set_frame_on(True)

axes[3][0].xaxis.set_label_coords(1, -0.175)
axes[2][0].yaxis.set_label_coords(-0.1, 1)
fig.subplots_adjust(wspace=0.025)
# plt.savefig("profiles.pdf", transparent=True, bbox_inches='tight', pad_inches=0)
# plt.close()
plt.show()

### Arrvial time surfaces

In [ ]:
#%%script false
# for i in range(lm.N_obj):
#     lm.obj_idx = i
i = objidx
arrival_time_surface_plot(lm, obj_index=i, draw_images=True,
                          contours=True, levels=60, min_contour_shift=None,
                          sad_contour_shift=None, scalebar=False, label=lm.obj_name, color='black')
# plt.plot(0, 0, color=gcl.blue, marker='+', markersize=16)
plot_scalebar(R=lm.maprad, length=max(int(lm.maprad/2), 1), color='black')
plt.axis('off')
plt.gcf().axes[0].get_xaxis().set_visible(False)
plt.gcf().axes[0].get_yaxis().set_visible(False)
plt.tight_layout()
plt.savefig('results/{}_{}_arriv.pdf'.format(lm.obj_name, lm.filename.replace('.state', '')), transparent=True, bbox_inches='tight', pad_inches=0)
# plt.close()
plt.show()


In [ ]:
%script false
print("Loading files...")

lm_H0 = LensModel(statedir+statefiles[7])
lm_H0.obj_idx = objidx
lm.obj_idx = objidx

print("Calculating arrival time grids...")
arriv = lm_H0.arrival_grid(model_index=-1)    # variable H0
arriv_fixed = lm.arrival_grid(model_index=-1) # CMB/SNeIa


In [ ]:
%%script false
# lm.obj_idx = objidx
# lm_H0.obj_idx = objidx
# arriv = lm_H0.arrival_grid(model_index=-1)
# arriv_fixed = lm.arrival_grid(model_index=-1)
tsqr2days = 365.25/lm.env.nu[0] * lm.env.objects[objidx].dL * (1+lm.env.objects[objidx].z)
print tsqr2days

delta = (arriv - arriv_fixed) * tsqr2days
print(delta.shape)
rmsk = radial_mask(delta, radius=37.5)
edge = np.abs(delta[rmsk].max())
vmin = -edge
vmax = edge
plt.imshow(delta, cmap='twilight', extent=lm.extent, origin='upper', vmin=vmin, vmax=vmax)
# plt.imshow(diffs, cmap='phoenix', extent=lm.extent, origin='upper', vmin=vmin, vmax=vmax)
plot_scalebar(R=lm.maprad, length=max(int(lm.maprad/2), 1), color='white')
plot_labelbox(objs[objidx], position='top right', padding=(0.03, 0.04), color='white', fontsize=14)
cbar = plt.colorbar()
cbar.ax.set_ylabel('[d]', rotation=0)
plt.axis('off')
plt.gcf().axes[0].get_xaxis().set_visible(False)
plt.gcf().axes[0].get_yaxis().set_visible(False)
plt.tight_layout()
plt.show()

In [ ]:
%%script false
limits = []
for i in range(len(objs)):
    lm.obj_idx = i
    lm_H0.obj_idx = i
    arriv = lm_H0.arrival_grid(model_index=-1)    # variable H0
    arriv_fixed = lm.arrival_grid(model_index=-1) # CMB/SNeIa

    tsqr2days = 365.25/lm.env.nu[0] * lm.env.objects[objidx].dL * (1+lm.env.objects[objidx].z)
    delta = (arriv - arriv_fixed) * tsqr2days
    rmsk = radial_mask(delta, radius=42.5)
    edge = np.abs(delta[rmsk]).max()
    vmin, vmax = -edge, edge
    cmapstr = 'twilight'
    plt.imshow(delta, cmap=cmapstr, extent=lm.extent, origin='upper', vmin=vmin, vmax=vmax)

    minima = lm.minima
    saddles = lm.saddle_points
    maxima = lm.maxima
    cmap = plt.get_cmap(cmapstr)
    min_clr, sad_clr, max_clr = cmap(.8), cmap(.5), cmap(.2)
    if len(minima) > 0:
        plt.plot(minima.T[0], minima.T[1], color=min_clr, marker='o', lw=0)
    if len(saddles) > 0:
        plt.plot(saddles.T[0], saddles.T[1], color=sad_clr, marker='o', lw=0)
    if len(maxima) > 0:
        plt.plot(maxima.T[0], maxima.T[1], color=max_clr, marker='o', lw=0)

    plot_scalebar(R=lm.maprad, length=max(int(lm.maprad/2), 1), color='white')
    plot_labelbox(objs[i], position='top right', padding=(0.03, 0.04), color='white', fontsize=14)
    cbar = plt.colorbar()
    cbar.ax.set_ylabel('[d]', rotation=0)
    plt.axis('off')
    plt.gcf().axes[0].get_xaxis().set_visible(False)
    plt.gcf().axes[0].get_yaxis().set_visible(False)
    plt.tight_layout()
    plt.savefig('results/{}_{}_arrivdelta.pdf'.format(lm.obj_name, lm.filename.replace('.state', '')),
                transparent=True, bbox_inches='tight', pad_inches=0)
    plt.show()
    plt.close()
